In [ ]:
# Vendo se temos uma GPU:
!nvidia-smi

Thu Aug 29 14:09:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import transformers

# pipeline serve para agrupar todas as funcoes necessarias da tarefa
# para processar a entrada e gerar a saida
modelo = transformers.pipeline(
    task='text-generation',
    model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    device='cuda' # cpu, cuda
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [5]:
# modelo escolhido: 22 blocos de processamento
# faz normalizacao
# suporta ate 2048 tokens de input
# embedding de 32000: ele conhece 32k palavras
# tarefa dele: recebe um input (texto que sera codificado em tokens com o tokenizer),
# e reornar a probabilidade de cada uma das 32k palavras de ser a proxima

modelo.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

In [12]:
def processa_prompt(modelo, prompt):
  print("="*80)

  # fazendo processo normla --- codificando
  tokens = modelo.tokenizer.encode(prompt, return_tensors='pt')

  print(f"- prompt original: {prompt}")
  print(f"- tokens: {tokens}")
  print(f"- # de tokens: {len(tokens)}")

  # fazendo o processo reverso --- decodificando
  decoded = modelo.tokenizer.convert_ids_to_tokens(tokens[0])
  print(f"- tokens decodificados: {decoded}")

  first_tokens = modelo.tokenizer.convert_ids_to_tokens(range(5))
  print(f"- primeiros tokens: {first_tokens}")

  # taquei o prompt pro LLM. temperatura controla o quao "criativo" o modelo sera
  # eh preciso setar do_sample para controlar a temperatura
  output = modelo(prompt, do_sample=True, temperature=0.7)
  print(f"- output do modelo: {output}")
  print(f"- texto gerado efetivamente: {output[0]['generated_text']}")

  print("="*80)
  return

processa_prompt(modelo, "Once upon a time")

- prompt original: Once upon a time
- tokens: tensor([[   1, 9038, 2501,  263,  931]])
- # de tokens: 1
- tokens decodificados: ['<s>', '▁Once', '▁upon', '▁a', '▁time']
- primeiros tokens: ['<unk>', '<s>', '</s>', '<0x00>', '<0x01>']
- output do modelo: [{'generated_text': "Once upon a time, there was a young girl named Lily who was struggling to find her place in the world. She had always been shy and quiet, preferring to keep to herself rather than make friends. She would spend her days reading books in a quiet corner of the library, lost in her own thoughts.\n\nOne day, Lily stumbled upon a book about fairies. She loved the idea of magical creatures and wondered if they were real. As she read more about them, she began to feel like she might see one. Suddenly, she saw a tiny, golden fairy flitting around a nearby pile of leaves. She smiled contentedly, delighted that she had found her spirit animal.\n\nAs Lily continued to explore, she met other fairy friends. They too were lost and